# Buscador de Atletas de Futebol por Características.
Por:
• Carlos Rafael Ferreira Góes
• Daniel Fonseca

A ideia desse projeto é realizar a raspagem de dados de jogadores de futebol do site Ogol, possibilitando assim a criação de um mecanismo que busque atletas através de suas características e estatísticas. Sua aplicação se dá no contexto de prospecção de jogadores e análise de mercado esportivo.

O Ogol, contém informações de mais de 990.000 jogadores, 57.000 treinadores e 120.000 equipes. Sendo uma plataforma colaborativa, ela está constantemente sendo atualizada e revisada, de modo que abrange praticamente todo o universo de atletas profissionais de alto rendimento. Uma característica do site, é a manutenção de todos os atletas sob o mesmo código, sua presença na url e a atualização de maneira crescente. Por exemplo, o primeiro atleta da base é o 1 enquanto o último jogador que acabou de ser adicionado é o 1204243. Portando a atualização da base para a entrada de novos jogadores se torna particularmente simples.

O trabalho também pode servir como facilitador para outras aplicações. Uma vez tendo acesso a essa base, uma gama de topicos de pesquisa se abrem, tanto na area esportiva, como em outras que podem usar o esporte como laboratorio. Como por exemplo estudos de gênero, raciais, pertencimento nacional, migração e etc.

Inicialmente, é necessário instalar um pacote, chamado selenium, que será essencial no acesso aos dados do site. Este é único pacote que de fato é requisitado na instalação. Na linha de código abaixo são apresentados os comandos para instalação no colab, assim como no jupyter (ambos como comentários). Obs: Os códigos para o jupyter devem ser executados no anaconda prompt.

In [ ]:
# !pip install selenium
# conda install selenium
# conda install -c conda-forge selenium

Abaixo, temos algumas importações necessárias para execução dos comandos. Algumas delas não são estritamente requisitadas, visto que ao longo do projeto encontrou-se maneiras mais eficientes de executar os códigos. Podemos destacar: o requests, que solicita a entrada no código fonte na obtenção dos dados almejados; o pandas, que refere-se a estrutura do dataframe; o webdriver (do pacote selenium), na estratégia utilizada para driblar o captcha; e por fim, o BeautifulSoup (do pacote bs4), que executa a obtenção dos dados.

In [2]:
import requests
import time
import pandas as pd
import os
# Utilizamos o webdriver para ultrapassar o Recaptcha
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

Após o carregamento dos pacotes acima, é descrito abaixo o site de interesse do presente projeto. Contudo, o site orientado pelo objeto abaixo url não está completo. Conforme mencionado anteriormente, o site possui um número de identificação para cada jogador. Deste modo, o url é descrito como a única parte em comum no endereço do site entre todos os(as) jogadores(as). Isto será fundamental na elaboração do loop mais adiante.

In [3]:
#url base do ogol
url = 'https://www.ogol.com.br/player.php?id='

Entretanto, o site ogol, assim como outros sites, tem um mecanismo de proteção à extração dos seus dados, o captcha. De modo que ao perceber um número elevado de tentativas de acesso ao site de um mesmo navegador em uma mesma sessão, a página trava o acesso ao código fonte impossibiitando á obtenção de dados por um determinado tempo. Portanto, a fim de resolver este problema foi realizado o download do chrome driver que permite executar o navegador chrome de modo automático com números de sessão diferentes. Deste modo, o captcha não consegue identificar que trata-se de um rôbo, e assim o acesso aos dados é livre de forma contínua. Ademais, como uma página do navegador chrome será executada à medida que é requerido o acesso a cada url do site, pode-se ser realizada algumas alterações no formato desta página com a função Options. Pode-se regular o tamanho da tela e, inclusive, executar a página apenas no background (chrome_options.add_argument("--headless")), sem que seja necessário dispender memória na exibição da tela.

In [4]:
# Chrome Headless
chrome_options = Options()
#chrome_options.add_argument("--headless")
# options = chrome_options,
driver = webdriver.Chrome(executable_path='C:/Users/crafa/Downloads/chromedriver')

Abaixo definimos as urls pesquisadas (lst). Nesse código deixamos apenas um demostrativo de como funciona essa busca. Enviamos em anexo uma amostra maior de mais de 20.000 jogadores. A compilação de toda base é possivel, contudo, diante do desempenho dos computadores utilizados associado à uma demanda de tempo elevada, optamos por mandar uma amostra da base.

s_new e dadost são listas que são criadas vazias, de modo que serão preenchidas na execução do loop mais adiante.

In [5]:
s_new = []
dadost = []
lst = list(range(211000,3+211000)) # Só colocar aqui o limite superior que temos todos os urls necessários

O Loop principal. Inicialmente, temos que informar que queremos entrar em cada página do site ogol a partir do id=1, e assim por diante. Sendo assim, s_new combina agora o objeto url descrito e cada número gerado na lst (como uma string). Assim, conseguimos obter o endereço dos sites e armazenamos para o acesso a posteriori.

Após isso, utilizamos o driver sobre as páginas geradas que estão em sites e solicitamos o acesso ao código fonte da página. Cria-se um dicionário vazio, onde serão armazedados as informações descritas de cada página.

Em seguida, a parte mais desafiadora do projeto ocorreu na identificação das tags de interesse com as informações das características dos atletas na montagem do loop. A utilização de uma determinada tag associada a um determinado atributo e uma posição não foi suficiente para encontrar algumas informações. Um exemplo representativo deste problema foi observado na obtenção da informação referente a posição que o atleta atua, pois alguns atletas variam em quantidades de informações. Por exemplo, atletas mais famosos têm registros de informações como: site pessoal, grau de parentesco e nome dos parentes, endereço das redes sociais e etc. Por outro lado, atletas menos badalados possuem menos informações. Deste modo, encontramos algumas divergências na posição da tag entre algumas páginas. Logo, era necessário uma abordagem diferente para identificar a tag de interesse de cada página. A solução encontrada foi identificar diretamente o conteúdo pretendido através da string "Posição" que aparecia dentro do código fonte. O mesmo foi executado para outras informações.

De maneira complementar, algumas informações não apresentavam uma string específica que se repetia no código fonte de cada página. Contudo, a posição era mesma para todas as páginas. Logo, informações como gols marcados, assistências, minutos, jogos, nome completo foram coletadas usando uma especificação mais simples.

Outro ponto importante, alguns urls são "não encontrados" (not found), ou seja, muito provavelmente já se referiram a algum atleta que foi excluido. Com isso, é imprescindível executar um pass dentro do except referente ao erro de índice (index error), pois desta forma entende-se que quando a página não for encontrada, a busca deve seguir para o url subsequente.

Por fim, alguns jogadores não têm determinados campos salvos. Daí marcamos essas informações como não encontradas para melhor track do sucesso da raspagem. Por outro lado, as informações obtidas foram todas armazenadas em um dicionário.

Obs: Inicialmente foi utilizando a função time.sleep do pacote time, mesmo para um intervalo de tempo de 15 segundos, o captcha em dado momento era acionado, impossibilitando a obtenção de dados dos/das atletas. Lembrando que o webdriver resolveu esta situação.

Obs2: Abaixo temos um exemplo apenas para demostração com 3 jogadores. A amostra coletada será inserida mais adiante.

In [6]:
for lst in lst:
    s_new = url + str(lst)
    sites = list(s_new.split("\n"))
    for sites in sites:
        try:
            driver.get(sites)
            #time.sleep(3)
            html_source = driver.page_source
            soup=BeautifulSoup(html_source,'lxml')
            dicio={}
            #gender
            gender = soup.find_all("div",attrs={"class":"info"})[0].text
            if "Feminino" in gender:
                genero = "Feminino"
                dicio['genero']=genero            
            else:
                genero = "Masculino"
                dicio['genero']=genero
            #position
            position_tag = soup.find('span', string='Posição')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    position = next_sibling.text.strip()
                    dicio['position']=position
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #full_name
            position_tag = soup.find('span', string='Nome')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    full_name = next_sibling.text.strip()
                    dicio['full_name']=full_name
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #birth_age
            position_tag = soup.find('span', string='Data de Nascimento/Idade')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    birth_age = next_sibling.text.strip()
                    dicio['birth_age']=birth_age
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #situation
            position_tag = soup.find('span', string='Situação')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    situation = next_sibling.text.strip()
                    dicio['situation']=situation
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #nationality
            position_tag = soup.find('span', string='Nacionalidade')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    nationality = next_sibling.text.strip()
                    dicio['nationality']=nationality
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #pe
            position_tag = soup.find('span', string='Pé preferencial')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    pe = next_sibling.text.strip()
                    dicio['pe']=pe
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #height
            position_tag = soup.find('span', string='Altura')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    height = next_sibling.text.strip()
                    dicio['height']=height
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #weight
            position_tag = soup.find('span', string='Peso')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    weight = next_sibling.text.strip()
                    dicio['weight']=weight
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #matches_national_team
            position_tag = soup.find('span', string='Jogos pela seleção')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    matches_national_team = next_sibling.text.strip()
                    dicio['matches_national_team']=matches_national_team
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #birth
            position_tag = soup.find('span', string='Naturalidade')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    birth = next_sibling.text.strip()
                    dicio['birth']=birth
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #club
            position_tag = soup.find('span', string='Clube atual')
            if position_tag:
                next_sibling = position_tag.find_previous('div')
                if next_sibling:
                    club = next_sibling.text.strip()
                    dicio['club']=club
                else:
                    print('Não encontrada')
            else:
                print('Não encontrada')
            #name
            name = soup.find_all("span",attrs={"class":"name"})[11].text
            dicio['name']=name
            #total_matches
            try:
                total_matches = soup.find_all("td",attrs={"class":"totals"})[1].text #Current season
            except IndexError:
                pass
            else:
                dicio['total_matches']=total_matches
            #minutes
            try:
                minutes = soup.find_all("td",attrs={"class":"totals"})[2].text #Current season
            except IndexError:
                pass
            else:        
                dicio['minutes']=minutes 
            #goals_scored
            try:
                goals_scored = soup.find_all("td",attrs={"class":"totals"})[3].text #Current season
            except IndexError:
                pass
            else:
                dicio['goals_scored']=goals_scored 
            #assistances
            try:
                assistances = soup.find_all("td",attrs={"class":"totals"})[4].text #Current season
            except IndexError:
                pass
            else:
                dicio['assistances']=assistances
            #code
            try:
                codigo = str(lst)
            except IndexError:
                pass
            else:
                dicio['codigo']=codigo
        except IndexError:
            pass
        else:
            dadost.append(dicio)


Não encontrada


Após a obtenção dos dados armazenados no dicionário de nome dadost, vamos criar um data frame com o pandas. Obs: Em virtude da quantidade de páginas disponíveis para extração de dados associada a limitada capacidade de processamento, as amostras foram exportadas em csv em diferentes computadores para a junção de uma amostra (a posteriori) para execução do buscador.

In [7]:
base_dados = pd.DataFrame(dadost)
# EXPORTAR ARQUIVO CSV: base_dados.to_csv("base_dados.csv",index=False)
base_dados

,birth,birth_age,club,codigo,full_name,genero,height,matches_national_team,name,nationality,pe,position,situation,weight
0,NaturalidadeSandavágur,Data de Nascimento/Idade1964-01-26(59 anos)Paí...,Clube atualSem Clube,211000,NomeTorkil Nielsen,Masculino,Altura0 cm,Jogos pela seleção16 Jogos / 1 Gols,Torkil Nielsen,NacionalidadeIlhas Faroé,Pé preferencial,PosiçãoMeia,SituaçãoAposentado,Peso0 kg
1,NaN,Data de Nascimento/Idade1975-11-25(47 anos)Paí...,Clube atualSem Clube,211001,NomeDanger Fourpence,Masculino,Altura0 cm,Jogos pela seleção,Danger Fourpence,NacionalidadeZimbabwe,Pé preferencial,PosiçãoDefensor,SituaçãoAposentado,Peso0 kg
2,NaturalidadeCape Town,Data de Nascimento/Idade1989-09-20(33 anos)Paí...,Clube atualSem Clube,211002,NomeBuyani Sali,Masculino,Altura0 cm,Jogos pela seleção,Buyani Sali,NacionalidadeÁfrica do Sul,Pé preferencial,PosiçãoMeia,SituaçãoAposentado,Peso0 kg


Após a obtenção dados, vamos tratá-los.

No tratamento de dados, o primeiro concerne na eliminação de algumas informações que estão juntas aos dados que não interessam. Depois vamos reescrever algumas informações que estão orientadas como string para serem transformadas em float (número). Pois, informações como altura, peso, idade e outras são identificadas numericamente.

In [5]:
# Tratando as informações
base_dados = base_dados.replace(['Nome', 'Naturalidade', 'Data de Nascimento/Idade', 'País de Nascimento', ' anos', 'Posição',
                         'Situação', 'Nacionalidade', 'Pé preferencial', 'Altura', 'cm', 'Peso', 'kg',
                         'Jogos pela seleção', 'Clube atual', 'NaN'], '', regex=True)

In [6]:
# Após o tratamento acima, segue:
base_dados['age'] = base_dados['birth_age'].str.extract(r'\((\d+)\)')

# Divide a coluna original pelo último parêntese e descarta o que estiver após ele
base_dados['birth_age'] = base_dados['birth_age'].str.rsplit('(', n=1, expand=True)[0]
base_dados['age']=base_dados['age'].astype(float)

In [ ]:
base_dados['height']=base_dados['height'].astype(float)
base_dados['weight']=base_dados['weight'].astype(float)
base_dados['assistances']=base_dados['assistances'].astype(float)
base_dados['goals_scored']=base_dados['goals_scored'].astype(float)
base_dados['minutes']=base_dados['minutes'].astype(float)
base_dados['total_matches']=base_dados['total_matches'].astype(float)

In [ ]:
# EXPORTAR ARQUIVO CSV após tratamento (se preferível): base_dados.to_csv("base_dados.csv",index=False)

Portanto, abaixo importamos os dados de diferentes arquivos csv e concatenamos em um único arquivo. Este arquivo será transformado em um data frame para iniciar o tratamento das informações.

In [ ]:
# Carregando os três arquivos CSV em DataFrames separados
# dados_rafael = pd.read_csv('C:/Users/crafa/Desktop/Dados ogol/dados_rafael.csv')
# dados_daniel = pd.read_csv('C:/Users/crafa/Desktop/Dados ogol/dados_daniel.csv')
# dados_carol = pd.read_csv('C:/Users/crafa/Desktop/Dados ogol/dados_carol.csv')

# Concatenando os DataFrames
# df_concatenado = pd.concat([dados_rafael,dados_daniel,dados_carol], ignore_index=True)
# Salvar o DataFrame concatenado em um novo arquivo CSV
# df_concatenado.to_csv('dados_ogol.csv', index=False)

Uma vez agrupado todos os dados em apenas um arquivo, vamos importar esta base de extensão csv para o Python.

In [11]:
base_dados = pd.read_csv('C:/Users/crafa/Desktop/Dados ogol/dados_ogol.csv')

In [12]:
base_dados

,assistances,birth,birth_age,club,codigo,full_name,genero,goals_scored,height,matches_national_team,minutes,name,nationality,pe,position,situation,total_matches,weight,age
0,NaN,NaN,1991-02-22,Sem Clube,200000.0,Benito Costanzo,Masculino,NaN,0.0,NaN,NaN,Benito Costanzo,Itália,NaN,Defensor,Aposentado,NaN,0.0,32.0
1,NaN,NaN,1984-01-13,Sem Clube,200001.0,Dario De Simone,Masculino,NaN,0.0,NaN,NaN,Dario De Simone,Itália,NaN,Atacante,Aposentado,NaN,0.0,39.0
2,NaN,Benevento,1978-03-07,Sem Clube,200002.0,Cristian Galliano,Masculino,NaN,176.0,NaN,NaN,Cristian Galliano,Itália,NaN,Defensor,Aposentado,NaN,70.0,45.0
3,NaN,Avellino,1973-07-01,Sem Clube,200003.0,Domenico Giugliano,Masculino,NaN,180.0,NaN,NaN,Domenico Giugliano,Itália,NaN,Meia,Aposentado,NaN,80.0,49.0
4,NaN,NaN,1990-00-00,Sem Clube,200004.0,Vincenzo Iorio,Masculino,NaN,0.0,NaN,NaN,Vincenzo Iorio,Itália,NaN,Defensor,Aposentado,NaN,0.0,32.0
5,NaN,NaN,1993-00-00,Sem Clube,200005.0,Gennaro Panariello,Masculino,NaN,0.0,NaN,NaN,Gennaro Panariello,Itália,NaN,Defensor,Aposentado,NaN,0.0,29.0
6,NaN,Mugnano di Napoli (NA),1991-06-20,Sem Clube,200006.0,Umberto Rupa,Masculino,NaN,0.0,NaN,NaN,Umberto Rupa,Itália,NaN,Meia,Aposentado,NaN,0.0,31.0
7,NaN,NaN,1986-06-19,Sem Clube,200007.0,Fabiano Sabatino,Masculino,NaN,184.0,NaN,NaN,Fabiano Sabatino,Itália,NaN,Defensor,Aposentado,NaN,78.0,36.0
8,NaN,NaN,1992-00-00,Sem Clube,200008.0,Moreno Triunfo,Masculino,NaN,0.0,NaN,NaN,Moreno Triunfo,Itália,NaN,Defensor,Aposentado,NaN,0.0,30.0
9,NaN,NaN,1991-06-20,Sem Clube,200009.0,Daniele Assene,Masculino,NaN,0.0,NaN,NaN,Daniele Assene,Itália,NaN,Meia,Aposentado,NaN,0.0,31.0


Então, com os dados tratados podemos iniciar a busca de atletas através de informações inderidas. A princípio temos um buscador (filtro) de informações bastante ingênuo através da função loc do pandas. Contudo, mais adiante será mencionado um buscador mais elaborado e funcional. No exemplo abaixo especificamos apenas quatro características de modo que são listados os(as) atletas conforme atribuições inseridas: Atletas em atividade, igual ou com mais de 1,90m de altura, com o pé dominante canhoto e que joga de meia.

In [24]:
# Exemplo de filtro
filtro_naive = base_dados.loc[(base_dados['situation'].str.contains('Em atividade',case=False))&
                        (base_dados['height']>=190) &
                        (base_dados['pe'].str.contains('Canhoto',case=False)) &
                        (base_dados['position'].str.contains('meia',case=False))]
                         
filtro_naive

,assistances,birth,birth_age,club,codigo,full_name,genero,goals_scored,height,matches_national_team,minutes,name,nationality,pe,position,situation,total_matches,weight,age
1987,NaN,Drogheda,1988-01-23,Boyne Rovers,202006.0,Gavin Brennan,Masculino,NaN,190.0,,NaN,Gavin Brennan,Rep. Irlanda,Canhoto,Meia / Defensor (Lateral Esquerdo),Em atividade,NaN,80.0,NaN
6772,NaN,Várzea - Barcelos,1992-05-30,Sem Clube,206912.0,Rui Filipe Pereira Faria,Masculino,NaN,194.0,,NaN,Rui Faria,Portugal,Canhoto,Defensor (Zagueiro) / Meia (Volante),Em atividade,NaN,87.0,NaN
8643,0,Vila Boim - Elvas,1991-10-24,Mosteirense,208888.0,João Filipe Covas Carapinha,Masculino,0.0,190.0,,1260,João Carapinha,Portugal,Canhoto,Meia (Volante) / Defensor (Zagueiro),Em atividade,14.0,77.0,NaN
10519,NaN,Rio de Janeiro,1993-10-08,Sem Clube,211715.0,Rudigullithi da Silva Henrique,Masculino,NaN,193.0,,NaN,Gullithi,Brasil,Canhoto,Defensor (Zagueiro) / Meia (Volante),Em atividade,NaN,94.0,NaN
11399,1,Sindelfingen,1994-05-13,Krylya Sovetov,212617.0,Amar Rahmanović,Masculino,2.0,192.0,5 Jogos / 0 Gols,1680,20.Amar Rahmanovic,Bósnia-HerzgovinaDupla Alemanha,Canhoto,Meia (Meia Ofensivo) / Meia (Meia Central),Em atividade,33.0,84.0,NaN


Com a finalidade de aprimorar nosso buscador, foi desenvolvido um buscador mais interativo e funcional que também permite a obtenção dos/das atletas de interesse de acordo com as características. Perguntas serão inseridas de modo que deve-se informar as características desejadas de acordo com a numeração que segue em ordem em consonância com a posição das características. Basta informar o número de acordo com a legenda e apertar ENTER ao final de cada pergunta.  

In [25]:
##criar opções

opcoes_posicao = ['','Atacante', 'Meia','Meia Ofensivo','Volante','Ponta','Ponta Esquerda', 'Ponta Direita', '2º Atacante','Defensor','Zagueiro', 'Lateral Esquerdo' , 'Lateral Direito', 'Goleiro']
opcoes_situacao = ['','Em atividade', 'Aposentado', 'Falecido']
opcoes_pe = ['','Destro', 'Canhoto']
opcoes_altura = ['','Maxima', 'Minima']
opcoes_peso = ['','Maximo', 'Minimo']
opcoes_selecao = ['','Sim', 'Não']
opcoes_idade = ['','Maximo', 'Minimo']
opcoes_partidas = ['','Maximo', 'Minimo']
opcoes_gols = ['','Maximo', 'Minimo']
opcoes_assistencias = ['','Maximo', 'Minimo']
opcoes_minutos = ['','Maximo', 'Minimo']
## Perguntas e aplicação dos filtros

#posição
print("Opções de filtro:")
print("1. Posição: ", opcoes_posicao)
posicao_opcao = int(input("Escolha uma opção para a posição de 1 a 13 (0 para tanto faz): ")) 
posicao = opcoes_posicao[posicao_opcao]
if posicao_opcao == 0:  # Tanto faz
    filtro = base_dados.loc[base_dados['position'] != 'zero ']
else: 
    filtro = base_dados.loc[base_dados['position'].str.contains(posicao)==True]
    
#situação
print("Opções de filtro:")
print("2. Situação: ", opcoes_situacao)
situacao_opcao = int(input("Escolha uma opção para a situação 1 a 3 (0 para tanto faz): ")) 
situacao = opcoes_situacao[situacao_opcao]
if situacao_opcao == 0:  # Tanto faz
    filtro2 = filtro.loc[filtro['situation'] != 'zero ']
else:
    filtro2 = filtro.loc[filtro['situation'].str.contains(situacao)==True]
    
#pe
print("Opções de filtro:")
print("3. Pé: ", opcoes_pe)
pe_opcao = int(input("Escolha uma opção para a situação 1 a 2 (0 para tanto faz): ")) 
pe=opcoes_pe[pe_opcao]
if pe_opcao == 0:  # Tanto faz
    filtro3 = filtro2.loc[filtro2['pe'] != 'zero ']
else:
    filtro3 = filtro2.loc[filtro2['pe'].str.contains(pe)==True] 
    
#nome
nome_completo = str(input("Digite um nome que aparece no nome completo (0 para tanto faz): "))
if nome_completo == '0':  # Tanto faz
    filtro4 = filtro3.loc[filtro3['full_name'] != 'zero ']
else:
    filtro4 = filtro3.loc[filtro3['full_name'].str.contains(nome_completo)==True]  
    
#nacionalidade
nacionalidade = str(input("Digite o país de nacionalidade (0 para tanto faz): "))
if nacionalidade == '0':  # Tanto faz
    filtro5 = filtro4.loc[filtro4['nationality'] != 'zero ']
else:
    filtro5 = filtro4.loc[filtro4['nationality'].str.contains(nacionalidade)==True]  
    
#altura
print("Opções de filtro:")
print("3. Altura: ", opcoes_altura)
altura_opcao1=int(input('Escolha uma opção para a altura de 1 a 2 (0 para tanto faz) :'))
if altura_opcao1 == 1:
    altura_opcao2=int(input('Qual altura em cm? :'))
    filtro6 = filtro5.loc[(filtro5['height']<= altura_opcao2) & (filtro5['height']> 0)]
if altura_opcao1 == 2:
    altura_opcao2=int(input('Qual altura em cm? :'))
    filtro6 = filtro5.loc[(filtro5['height']>= altura_opcao2)& (filtro5['height']> 0)]
if altura_opcao1 == 0:
    filtro6 = filtro5.loc[(filtro5['height']!= 'zero')]

#peso
print("Opções de filtro:")
print("3. Peso: ", opcoes_peso)
peso_opcao1=int(input('Escolha uma opção para a peso de 1 a 2 (0 para tanto faz) :'))
if peso_opcao1 == 1:
    peso_opcao2=int(input('Qual peso em kg? :'))
    filtro7 = filtro6.loc[(filtro6['weight']<= peso_opcao2) & (filtro6['weight']> 0)]
if peso_opcao1 == 2:
    peso_opcao2=int(input('Qual peso em kg? :'))
    filtro7 = filtro6.loc[(filtro6['weight']>= peso_opcao2)& (filtro6['weight']> 0)]
if peso_opcao1 == 0:
    filtro7 = filtro6.loc[(filtro6['weight']!= 'zero')]

#clube atual
clube = str(input("Digite o clube nessa temporada (0 para tanto faz): "))
if clube == '0':  # Tanto faz
    filtro8 = filtro7.loc[filtro7['club'] != 'zero ']
else:
    filtro8 = filtro7.loc[filtro7['club'].str.contains(clube)==True]  
    
#nome
nome_conhecido = str(input("Digite um nome que aparece no nome que ele é conhecido (0 para tanto faz): "))
if nome_conhecido == '0':  # Tanto faz
    filtro9 = filtro8.loc[filtro8['name'] != 'zero ']
else:
    filtro9 = filtro8.loc[filtro8['name'].str.contains(nome_conhecido)==True]  

#partidas
print("Opções de filtro:")
print("3. Partidas: ", opcoes_partidas)
partidas_opcao1=int(input('Escolha uma opção para a partidas nessa temporada de 1 a 2 (0 para tanto faz) :'))
if partidas_opcao1 == 1:
    partidas_opcao2=int(input('Quantidade de partidas :'))
    filtro10 = filtro9.loc[(filtro9['total_matches']<= partidas_opcao2) & (filtro9['total_matches']> 0)]
if partidas_opcao1 == 2:
    partidas_opcao2=int(input('Quantidade de partidas :'))
    filtro10 = filtro9.loc[(filtro9['total_matches']>= partidas_opcao2)& (filtro9['total_matches']> 0)]
if partidas_opcao1 == 0:
    filtro10 = filtro9.loc[(filtro9['total_matches']!= 'zero')]

#minutos
print("Opções de filtro:")
print("3. Minutos: ", opcoes_minutos)
minutos_opcao1=int(input('Escolha uma opção para minutos nessa temporada de 1 a 2 (0 para tanto faz) :'))
if minutos_opcao1 == 1:
    minutos_opcao2=int(input('Quantidade de minutos :'))
    filtro11 = filtro10.loc[(filtro10['minutes']<= minutos_opcao2) & (filtro10['minutes']> 0)]
if minutos_opcao1 == 2:
    minutos_opcao2=int(input('Quantidade de minutos :'))
    filtro11 = filtro10.loc[(filtro10['minutes']>= minutos_opcao2)& (filtro10['minutes']> 0)]
if minutos_opcao1 == 0:
    filtro11 = filtro10.loc[(filtro10['minutes']!= 'zero')]
    
#gols
print("Opções de filtro:")
print("3. Gols: ", opcoes_gols)
gols_opcao1=int(input('Escolha uma opção para gols nessa temporada de 1 a 2 (0 para tanto faz) :'))
if gols_opcao1 == 1:
    gols_opcao2=int(input('Quantidade de gols :'))
    filtro12 = filtro11.loc[(filtro11['goals_scored']<= gols_opcao2) & (filtro11['goals_scored']> 0)]
if gols_opcao1 == 2:
    gols_opcao2=int(input('Quantidade de gols :'))
    filtro12 = filtro11.loc[(filtro11['goals_scored']>= gols_opcao2)& (filtro11['goals_scored']> 0)]
if gols_opcao1 == 0:
    filtro12 = filtro11.loc[(filtro11['goals_scored']!= 'zero')] 
    
#assistencias
print("Opções de filtro:")
print("3. Assistencias: ", opcoes_assistencias)
assistencias_opcao1=int(input('Escolha uma opção para assistencias nessa temporada de 1 a 2 (0 para tanto faz) :'))
if assistencias_opcao1 == 1:
    assistencias_opcao2=int(input('Quantidade de assistencias :'))
    filtro13 = filtro12.loc[(filtro12['assistances']<= assistencias_opcao2) & (filtro12['assistances']> 0)]
if assistencias_opcao1 == 2:
    assistencias_opcao2=int(input('Quantidade de gols :'))
    filtro13 = filtro12.loc[(filtro12['assistances']>= assistencias_opcao2)& (filtro12['assistances']> 0)]
if assistencias_opcao1 == 0:
    filtro13 = filtro12.loc[(filtro12['assistances']!= 'zero')] 
    
#idade
print("Opções de filtro:")
print("3. Idade: ", opcoes_idade)
idade_opcao1=int(input('Escolha uma opção para idade nessa temporada de 1 a 2 (0 para tanto faz) :'))
if idade_opcao1 == 1:
    idade_opcao2=int(input('idade :'))
    filtro14 = filtro13.loc[(filtro13['age']<= idade_opcao2) & (filtro13['age']> 0)]
if idade_opcao1 == 2:
    idade_opcao2=int(input('idade :'))
    filtro14 = filtro13.loc[(filtro13['age']>= idade_opcao2)& (filtro13['age']> 0)]
if idade_opcao1 == 0:
    filtro14 = filtro13.loc[(filtro13['age']!= 'zero')]     
    
#selecao
print("Opções de filtro:")
print("3. Já jogou na seleção : ", opcoes_selecao)
selecao_opcao1=int(input('Escolha uma opção para seleçao de 1 a 2 (0 para tanto faz) :'))
if selecao_opcao1 == 1:
    filtro15 = filtro14.loc[(filtro14['matches_national_team'].str.contains('Jogos')==True)] 
if selecao_opcao1 == 2:
    filtro15 = filtro14.loc[(filtro14['matches_national_team'].str.contains('Jogos')==False)]
if selecao_opcao1 == 0:
    filtro15 = filtro14.loc[(filtro14['matches_national_team']!= 'zero')] 
    
## Exibir os resultados
filtro15

Opções de filtro:
1. Posição:  ['', 'Atacante', 'Meia', 'Meia Ofensivo', 'Volante', 'Ponta', 'Ponta Esquerda', 'Ponta Direita', '2º Atacante', 'Defensor', 'Zagueiro', 'Lateral Esquerdo', 'Lateral Direito', 'Goleiro']
Escolha uma opção para a posição de 1 a 13 (0 para tanto faz): 2
Opções de filtro:
2. Situação:  ['', 'Em atividade', 'Aposentado', 'Falecido']
Escolha uma opção para a situação 1 a 3 (0 para tanto faz): 1
Opções de filtro:
3. Pé:  ['', 'Destro', 'Canhoto']
Escolha uma opção para a situação 1 a 2 (0 para tanto faz): 2
Digite um nome que aparece no nome completo (0 para tanto faz): 0
Digite o país de nacionalidade (0 para tanto faz): 0
Opções de filtro:
3. Altura:  ['', 'Maxima', 'Minima']
Escolha uma opção para a altura de 1 a 2 (0 para tanto faz) :2
Qual altura em cm? :190
Opções de filtro:
3. Peso:  ['', 'Maximo', 'Minimo']
Escolha uma opção para a peso de 1 a 2 (0 para tanto faz) :0


C:\Users\crafa\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Digite o clube nessa temporada (0 para tanto faz): 0
Digite um nome que aparece no nome que ele é conhecido (0 para tanto faz): 0
Opções de filtro:
3. Partidas:  ['', 'Maximo', 'Minimo']
Escolha uma opção para a partidas nessa temporada de 1 a 2 (0 para tanto faz) :0
Opções de filtro:
3. Minutos:  ['', 'Maximo', 'Minimo']
Escolha uma opção para minutos nessa temporada de 1 a 2 (0 para tanto faz) :0
Opções de filtro:
3. Gols:  ['', 'Maximo', 'Minimo']
Escolha uma opção para gols nessa temporada de 1 a 2 (0 para tanto faz) :0
Opções de filtro:
3. Assistencias:  ['', 'Maximo', 'Minimo']
Escolha uma opção para assistencias nessa temporada de 1 a 2 (0 para tanto faz) :0
Opções de filtro:
3. Idade:  ['', 'Maximo', 'Minimo']
Escolha uma opção para idade nessa temporada de 1 a 2 (0 para tanto faz) :0
Opções de filtro:
3. Já jogou na seleção :  ['', 'Sim', 'Não']
Escolha uma opção para seleçao de 1 a 2 (0 para tanto faz) :0


,assistances,birth,birth_age,club,codigo,full_name,genero,goals_scored,height,matches_national_team,minutes,name,nationality,pe,position,situation,total_matches,weight,age
1987,NaN,Drogheda,1988-01-23,Boyne Rovers,202006.0,Gavin Brennan,Masculino,NaN,190.0,,NaN,Gavin Brennan,Rep. Irlanda,Canhoto,Meia / Defensor (Lateral Esquerdo),Em atividade,NaN,80.0,NaN
6772,NaN,Várzea - Barcelos,1992-05-30,Sem Clube,206912.0,Rui Filipe Pereira Faria,Masculino,NaN,194.0,,NaN,Rui Faria,Portugal,Canhoto,Defensor (Zagueiro) / Meia (Volante),Em atividade,NaN,87.0,NaN
8643,0,Vila Boim - Elvas,1991-10-24,Mosteirense,208888.0,João Filipe Covas Carapinha,Masculino,0.0,190.0,,1260,João Carapinha,Portugal,Canhoto,Meia (Volante) / Defensor (Zagueiro),Em atividade,14.0,77.0,NaN
10519,NaN,Rio de Janeiro,1993-10-08,Sem Clube,211715.0,Rudigullithi da Silva Henrique,Masculino,NaN,193.0,,NaN,Gullithi,Brasil,Canhoto,Defensor (Zagueiro) / Meia (Volante),Em atividade,NaN,94.0,NaN
11399,1,Sindelfingen,1994-05-13,Krylya Sovetov,212617.0,Amar Rahmanović,Masculino,2.0,192.0,5 Jogos / 0 Gols,1680,20.Amar Rahmanovic,Bósnia-HerzgovinaDupla Alemanha,Canhoto,Meia (Meia Ofensivo) / Meia (Meia Central),Em atividade,33.0,84.0,NaN


Para exemplificar, as mesmas características requeridas no buscador mais ingênuo foram repetidas, e o resultado é o mesmo.